# California Grape Stree villas
https://www.crexi.com/properties/360888/california-grape-street-villas?crexi_url_type=2

# over all
was an air bnb, but vacancy rates are high and now turning to a sell. trying to sell but how to value. 

In [2]:
import numpy as np
import scipy as sp
import pandas as pd
import datetime

from compute import *

In [13]:
def computeProjectSimple(projectName, numberUnits, price, rent, expenseRate, ltv, loanRatePer, rentPF,expenseRatePF,capPF):
    debtPmtPerYr = np.pmt(loanRatePer/100/12., 360,price*ltv/100.,)*-12
    noi=computeNoi(rent,expenseRate)    
    noiPF=computeNoi(rentPF,expenseRatePF)    
    pricePF=noiPF/(capPF/100.)

    print("""
    Project   : {Project}
    Purchase  : ${Price:,}
    rent roll : ${Rent:,}
    NOI       : ${Noi:,.0f}  
    Expen Rate: {ExpRatio:.0f}%
    Cap Rate  : {Cap:.2f}%        
    DebService:
       amount - ${LoanV:,.0f}
       rate   - {LoanRatePer}%
       payment- {Payment:,.0f}$
    DSCR      : {Dscr:2.2f}%  
    """.format(
        Project=ProjectName,
        Price=price,
        Rent=rent *12,
        Cap = computeCap(rent,price, expenseRate),
        Noi = noi,
        ExpRatio=expenseRate,
        LoanV = price*ltv/100.,
        Payment=debtPmtPerYr,
        Dscr=noi / debtPmtPerYr,
        LoanRatePer=loanRatePer
         ))
    print("""    
    rent roll : ${Rent:,.0f}
    NOI       : ${Noi:,.0f}
    Expen Rate: {ExpRatio}%    
    Cap Rate  : {Cap:.2f}%        
    Price     : ${Price:,.0f} @ cap({CapPF:.2f}%)
    """.format(            
        Rent=rentPF*12,
        Cap = computeCap(rentPF,price, expenseRatePF),
        Noi = noiPF,
        ExpRatio=expenseRatePF,
        CapPF=capPF,
        Price=pricePF))

In [14]:
# long term rental

#constants
ProjectName = "California Grape Street villas"
# assumptions
NumberUnits=16
price     = 3700000
rent      = 1700*12+1300*4
ltv       = 75
loanRatePer = 4
expenseRate =35
rentPF= 2000*12+1500*4
expenseRatePF=30
capPF=4.5
repairCost  = NumberUnits*15000
#computeProjectSimple(projectName, numberUnits, price, rent, expenseRate, ltv, loanRatePer, rentPf,expenseRatePf,capPF):
computeProjectSimple(ProjectName, NumberUnits, price, rent,expenseRate, ltv, loanRatePer, rentPF,expenseRatePF,capPF)




SquareFootage=6852
LotSize =5020
NumBds = ['1bd1ba'*14,'studio'*2]


    Project   : California Grape Street villas
    Purchase  : $3,700,000
    rent roll : $307,200
    NOI       : $199,680  
    Expen Rate: 35%
    Cap Rate  : 5.40%        
    DebService:
       amount - $2,775,000
       rate   - 4%
       payment- 158,979$
    DSCR      : 1.26%  
    
    
    rent roll : $360,000
    NOI       : $252,000
    Expen Rate: 30%    
    Cap Rate  : 6.81%        
    Price     : $5,600,000 @ cap(4.50%)
    


In [15]:
dfAct = pd.DataFrame(columns=["Value","Monthly","Annual","%"])

dfAct.loc["Purchase Price","Value"] = price
dfAct.loc["closing cost","%"]       = 2
dfAct.loc["Repair","Value"]         = 100000
dfAct.loc["Reserve","Value"]        = 80000
dfAct.loc["Rent Roll","Monthly"] = rent
dfAct.loc["Loan Rate","%"]  = 4
dfAct.loc["LTV","%"]        = 75
dfAct.loc["Number Of Units","Value"]=16
simpleExpense = 0
dfAct.loc["Util Per Unit","Value"]  =100
dfAct.loc["Insurance","%"]  =0.2
dfAct.loc["Expenses","%"]   =35
dfAct.loc["Taxes","%"]      = 1.25
dfAct.loc["Vacancy","%"]    = 4
dfAct.loc["Management","%"] = 5
dfAct.loc["Maintenance","%"]= 0.3
## derived
#NOI 

dfact = setMonthlyAndAnnual(dfAct)
if simpleExpense ==1:
    noi=computeNoi(rent=dfAct.loc["Rent Roll","Monthly"],expenseRate=dfAct.loc["Expense Rate","%"]/100)
    dfAct.loc["NOI","Annual"] = noi
    dfAct.loc["Expenses","Monthly"]=dfAct.loc["Expenses","%"] * dfAct.loc["Rent Roll","Monthly"],
else:
    dfAct.loc["Util","Monthly"] = dfAct.loc["Number Of Units","Value"] * dfAct.loc["Util Per Unit","Value"]
    for c in ["Vacancy","Management"]:
        dfAct.loc[c,"Monthly"] = dfAct.loc[c,"%"]/100.*dfAct.loc["Rent Roll","Monthly"]
        dfAct.loc[c,"Monthly"] = dfAct.loc[c,"%"]/100.*dfAct.loc["Rent Roll","Monthly"]
    for c in ["Maintenance","Taxes","Insurance"]:
        dfAct.loc[c,"Monthly"]= dfAct.loc[c,"%"]/100.*dfAct.loc["Purchase Price","Value"]/12.
    dfAct.loc["Expenses","Monthly"] = sum(dfAct.loc[["Vacancy","Management","Maintenance","Taxes","Insurance","Util"],"Monthly"])
    dfAct.loc["NOI","Monthly"]      = dfAct.loc["Rent Roll","Monthly"] - dfAct.loc["Expenses","Monthly"]
    dfAct.loc["Expense","%"] = dfAct.loc["Expenses","Monthly"]/ dfAct.loc["Rent Roll","Monthly"]
    
dfAct.loc["closing cost","Value"]=dfAct.loc["Purchase Price","Value"] * dfAct.loc["closing cost","%"]/100.
dfAct.loc["Cap","%"] = computeCap(
    rent       =dfAct.loc["Rent Roll","Monthly"],
    price      =+dfAct.loc["Repair","Value"],
    expenses=dfAct.loc["Expenses","Monthly"]/100)    

# loan payment
pmt =  -1*np.pmt(
    dfAct.loc["Loan Rate","%"]/100/12.,
    360,dfAct.loc["Purchase Price","Value"] *dfAct.loc["LTV","%"]/100)
dfAct.loc["Debt Service","Monthly"] = pmt
dfAct.loc["DSCR","%"] = dfAct.loc["NOI","Monthly"] / dfAct.loc["Debt Service","Monthly"] 
dfact = setMonthlyAndAnnual(dfAct)

dfAct.loc["Cash Flow","Annual"] = (dfAct.loc["NOI","Annual"] - dfAct.loc["Debt Service","Annual"]) 

dfAct.loc["Out Of Pocket","Value"]=\
    dfAct.loc["closing cost","Value"]+\
    dfAct.loc["Repair","Value"]+\
    dfAct.loc["Reserve","Value"]+\
    dfAct.loc["Purchase Price","Value"]*(100-dfAct.loc["LTV","%"])/100.
dfAct.loc["Cash Flow","%"] = dfAct.loc["Cash Flow","Annual"] / dfAct.loc["Out Of Pocket","Value"] * 100
dfact = setMonthlyAndAnnual(dfAct)
dfAct

,Value,Monthly,Annual,%
Purchase Price,3700000,NaN,NaN,NaN
closing cost,74000,NaN,NaN,2
Repair,100000,NaN,NaN,NaN
Reserve,80000,NaN,NaN,NaN
Rent Roll,NaN,25600,307200,NaN
Loan Rate,NaN,NaN,NaN,4
LTV,NaN,NaN,NaN,75
Number Of Units,16,NaN,NaN,NaN
Util Per Unit,100,NaN,NaN,NaN
Insurance,NaN,616.667,7400,0.2


In [16]:
simpleDepreciation=1
rentIncrease=5        # rent increase
maintenanceIncrease=3 # for all OI
years =datetime.datetime.today().year + np.arange(0,10)
dfPF = pd.DataFrame(columns=years)

for year in years:
    if year == years[0]:
        dfPF.loc["Rent Roll",year]    = dfAct.loc["Rent Roll","Annual"]
        dfPF.loc["Expenses", year]    = -1*dfAct.loc["Expenses","Annual"]
    else:
        dfPF.loc["Rent Roll",year]    = dfPF.loc["Rent Roll",year-1] *(100+rentIncrease)/100.
        dfPF.loc["Expenses",year]     = dfPF.loc["Expenses",year-1]  *(100+maintenanceIncrease)/100.
        
    dfPF.loc["NOI",year]          = dfPF.loc["Rent Roll",year]+dfPF.loc["Expenses",year]
    dfPF.loc["Debt Service",year] = computePmt(dfAct,year=1)*12
    if simpleDepreciation == 1:
        dfPF.loc["Depreciation",year] = (dfAct.loc["Repair","Value"]+dfAct.loc["Purchase Price","Value"])* -2./3./27.5    
    # Cash Flow NOI minus debt service
    dfPF.loc["Cash Flow",year]    = dfPF.loc["NOI",year] + dfPF.loc["Debt Service",year]
    # Profit=NOI minus interest payment
    dfPF.loc["Profit",year]       = dfPF.loc["NOI",year] + computePmt(dfAct,year=1,ptype=LoanPaymentType.Interest)*12    
    # Pre Tax Profit = profit - minus depreciation
    dfPF.loc["Pre-Tax Profit",year]   = dfPF.loc["Profit",year] + dfPF.loc["Depreciation",year]
    dfPF.loc["Cap Rate",year]         = dfPF.loc["NOI",year]      / dfAct.loc["Purchase Price","Value"]*100
    # cash on cash = profit / out of pocket acquisition
    dfPF.loc["Cash on Cash",year]     = dfPF.loc["Profit",year]/ dfAct.loc["Out Of Pocket","Value"] * 100
    #dfPF.loc["Cash Flow","%"] = dfPF.loc["Cash Flow","Annual"] / dfAct.loc["Out Of Pocket","Value"] * 100
dfPF

,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029
Rent Roll,307200,322560,338688,355622,373404,392074,411677,432261,453874,476568
Expenses,-111598,-114946,-118394,-121946,-125605,-129373,-133254,-137251,-141369,-145610
NOI,195602,207614,220294,233676,247799,262701,278424,295010,312505,330958
Debt Service,-158979,-158979,-158979,-158979,-158979,-158979,-158979,-158979,-158979,-158979
Depreciation,-92121.2,-92121.2,-92121.2,-92121.2,-92121.2,-92121.2,-92121.2,-92121.2,-92121.2,-92121.2
Cash Flow,36622.7,48634.8,61314.4,74697,88819.7,103722,119444,136030,153526,171979
Profit,84602,96614.1,109294,122676,136799,151701,167424,184010,201505,219958
Pre-Tax Profit,-7519.21,4492.85,17172.5,30555,44677.8,59579.8,75302.3,91888.6,109384,127837
Cap Rate,5.28654,5.61119,5.95388,6.31557,6.69727,7.10003,7.52496,7.97324,8.44609,8.94481
Cash on Cash,7.17574,8.19458,9.27003,10.4051,11.603,12.8669,14.2005,15.6073,17.0912,18.6563
